Some very good link to follow:

https://stackoverflow.com/questions/34948296/using-pyspark-to-connect-to-postgresql



In [1]:
import psycopg2
import getpass
import pandas as pd

In [2]:
# input password

print("Enter password:")
password = getpass.getpass()

Enter password:
········


In [3]:
# Note: please change your database, username & password as per your own values
conn_params_dic = {
    "host"      : "localhost",
    "port"      : "5432",
    "database"  : "mydb",
    "user"      : "postgres",
    "password"  : password
}

In [4]:
# Load data

irisData = pd.read_csv('data/iris.csv')

In [5]:
# fit SQL format

irisData['variety'] = irisData.variety.apply(lambda x: "'"+x+"'")

In [6]:
irisData

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,'Setosa'
1,4.9,3.0,1.4,0.2,'Setosa'
2,4.7,3.2,1.3,0.2,'Setosa'
3,4.6,3.1,1.5,0.2,'Setosa'
4,5.0,3.6,1.4,0.2,'Setosa'
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,'Virginica'
146,6.3,2.5,5.0,1.9,'Virginica'
147,6.5,3.0,5.2,2.0,'Virginica'
148,6.2,3.4,5.4,2.3,'Virginica'


In [7]:
print('Connecting to the PostgreSQL...')
conn = psycopg2.connect(**conn_params_dic)
#conn.autocommit = True
print("Connection successful...")

Connecting to the PostgreSQL...
Connection successful...


In [8]:
#cursor = conn.cursor();
# Dropping table iris if exists
#cursor.execute("DROP TABLE IF EXISTS iris;")
        
#sql = '''CREATE TABLE "iris"(
#sepal_length DECIMAL(2,1) NOT NULL, 
#sepal_width DECIMAL(2,1) NOT NULL, 
#petal_length DECIMAL(2,1) NOT NULL, 
#petal_width DECIMAL(2,1),
#species CHAR(11)NOT NULL
#)'''
        
# Creating a table
#cursor.execute(sql);
#print("iris table is created successfully.")
    
# Closing the cursor & connection
#cursor.close()

In [9]:
iris_tuples = list(irisData.itertuples(index=False, name=None))
iris_tuples_string = ",".join(["(" + ",".join([str(w) for w in wt]) + ")" for wt in iris_tuples])

iris_tuples_string

"(5.1,3.5,1.4,0.2,'Setosa'),(4.9,3.0,1.4,0.2,'Setosa'),(4.7,3.2,1.3,0.2,'Setosa'),(4.6,3.1,1.5,0.2,'Setosa'),(5.0,3.6,1.4,0.2,'Setosa'),(5.4,3.9,1.7,0.4,'Setosa'),(4.6,3.4,1.4,0.3,'Setosa'),(5.0,3.4,1.5,0.2,'Setosa'),(4.4,2.9,1.4,0.2,'Setosa'),(4.9,3.1,1.5,0.1,'Setosa'),(5.4,3.7,1.5,0.2,'Setosa'),(4.8,3.4,1.6,0.2,'Setosa'),(4.8,3.0,1.4,0.1,'Setosa'),(4.3,3.0,1.1,0.1,'Setosa'),(5.8,4.0,1.2,0.2,'Setosa'),(5.7,4.4,1.5,0.4,'Setosa'),(5.4,3.9,1.3,0.4,'Setosa'),(5.1,3.5,1.4,0.3,'Setosa'),(5.7,3.8,1.7,0.3,'Setosa'),(5.1,3.8,1.5,0.3,'Setosa'),(5.4,3.4,1.7,0.2,'Setosa'),(5.1,3.7,1.5,0.4,'Setosa'),(4.6,3.6,1.0,0.2,'Setosa'),(5.1,3.3,1.7,0.5,'Setosa'),(4.8,3.4,1.9,0.2,'Setosa'),(5.0,3.0,1.6,0.2,'Setosa'),(5.0,3.4,1.6,0.4,'Setosa'),(5.2,3.5,1.5,0.2,'Setosa'),(5.2,3.4,1.4,0.2,'Setosa'),(4.7,3.2,1.6,0.2,'Setosa'),(4.8,3.1,1.6,0.2,'Setosa'),(5.4,3.4,1.5,0.4,'Setosa'),(5.2,4.1,1.5,0.1,'Setosa'),(5.5,4.2,1.4,0.2,'Setosa'),(4.9,3.1,1.5,0.2,'Setosa'),(5.0,3.2,1.2,0.2,'Setosa'),(5.5,3.5,1.3,0.2,'Setosa'),

In [10]:
# Read data into iris

cursor = conn.cursor();
# Dropping table iris if exists
cursor.execute("DROP TABLE IF EXISTS iris;")
        
sql = '''CREATE TABLE iris(
sepal_length DECIMAL(2,1) NOT NULL, 
sepal_width DECIMAL(2,1) NOT NULL, 
petal_length DECIMAL(2,1) NOT NULL, 
petal_width DECIMAL(2,1),
species CHAR(11)NOT NULL
)'''
        
# Creating a table
cursor.execute(sql);
cursor.execute("INSERT INTO iris(sepal_length, sepal_width, petal_length, \
                   petal_width, species) VALUES " + iris_tuples_string + ";")
cursor.execute('commit')

#cursor.execute("FLUSH TABLES;")

# close the cursor object to avoid memory leaks
cursor.close()

# close the connection object
conn.close()

In [11]:
# Load the table

conn = psycopg2.connect(**conn_params_dic)
# declare a cursor object from the connection
cursor = conn.cursor()

# Execute query
sql = "SELECT * FROM iris"
cursor.execute(sql)

# Fetch all the records
tuples = cursor.fetchall()

# Preparing list of columns for dataframe

cols = list(irisData.columns)
irisdf = pd.DataFrame(tuples,columns=cols)

print(irisdf.head())

# close the cursor object to avoid memory leaks
#cursor.close()

# close the connection object
#conn.close()

  sepal.length sepal.width petal.length petal.width      variety
0          5.1         3.5          1.4         0.2  Setosa     
1          4.9         3.0          1.4         0.2  Setosa     
2          4.7         3.2          1.3         0.2  Setosa     
3          4.6         3.1          1.5         0.2  Setosa     
4          5.0         3.6          1.4         0.2  Setosa     


In [12]:
# close the cursor object to avoid memory leaks
#cursor.close()

# close the connection object
#conn.close()

In [13]:
from pyspark.sql import SparkSession

In [14]:
#spark = SparkSession.builder.config("spark.jars", "/Applications/dsdriver/java/postgresql-42.2.20.jar").getOrCreate()

In [15]:
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.jars", "/Applications/dsdriver/java/postgresql-42.2.20.jar") \
    .getOrCreate()

In [16]:
df = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/mydb") \
    .option("dbtable", "iris") \
    .option("user", "postgres") \
    .option("password", password) \
    .option("driver", "org.postgresql.Driver") \
    .load()

In [17]:
from pyspark.ml.feature import VectorAssembler

train_df, test_df = df.randomSplit([.8, .2], seed=12345)
predictors = ["sepal_length", "sepal_width", "petal_length", "petal_width"]
vec_assembler = VectorAssembler(inputCols=predictors, outputCol="features")
vec_train_df = vec_assembler.transform(train_df)
vec_train_df.select("features", "species").show(5)

+-----------------+-----------+
|         features|    species|
+-----------------+-----------+
|[4.3,3.0,1.1,0.1]|Setosa     |
|[4.4,2.9,1.4,0.2]|Setosa     |
|[4.4,3.0,1.3,0.2]|Setosa     |
|[4.4,3.2,1.3,0.2]|Setosa     |
|[4.5,2.3,1.3,0.3]|Setosa     |
+-----------------+-----------+
only showing top 5 rows



In [18]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(labelCol="species", featuresCol="features")
lr_model = lr.fit(vec_train_df)
vec_test_df = vec_assembler.transform(test_df)
predictions = lr_model.transform(vec_test_df)

IllegalArgumentException: requirement failed: Column species must be of type numeric but was actually of type string.